In [ ]:
!pip install transformers torch
!pip install accelerate -U

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

torch.device("cpu")

In [ ]:
# Load dataset
df = pd.read_csv('data_2000_rows.csv') 
labels = pd.read_csv('labels_from_1_to_1000.csv') 

df = df.drop('label',axis=1)
df = df.drop('Unnamed: 0',axis=1)
df = df.drop('customer_unique_id',axis=1)
df = df.drop('date',axis=1)
df = df.drop('row_count',axis=1)
df = df.head(1000)
labels = labels.rename( columns = {'9' : 'label'})
df = df.rename( columns = {'conversation' : 'text'})
df['label'] = labels['label'].copy()
print(df.info())
print(df.head())

In [ ]:
# Map labels to start from 0
label_mapping = {label: label + 1 for label in df['label'].unique()}
df['label'] = df['label'].map(label_mapping)
# df['label'] = df['label'].replace(-1, 0)
print(df.info())

In [ ]:
# Split the dataset into training and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
# Define a custom dataset class
class ConversationDataset(Dataset):
    def __init__(self, text_list, label_list, tokenizer, max_length):
        self.text_list = text_list
        self.label_list = label_list
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.text_list)

    def __getitem__(self, idx):
        text = str(self.text_list[idx])
        label = int(self.label_list[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_length,
            padding='max_length',
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# Load the pre-trained Turkish BERT model and tokenizer
model_name = 'bert-files'
tokenizer = BertTokenizer.from_pretrained(model_name)
num_labels = len(df['label'].unique())
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [ ]:
# Define the dataset and dataloaders
train_dataset = ConversationDataset(train_df['text'].tolist(), train_df['label'].tolist(), tokenizer, max_length=512)
val_dataset = ConversationDataset(val_df['text'].tolist(), val_df['label'].tolist(), tokenizer, max_length=512)


train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average='weighted')
    accuracy = accuracy_score(labels, preds)
    return {'f1': f1, 'accuracy': accuracy}

In [ ]:
# Define training arguments and instantiate Trainer
training_args = TrainingArguments(
    output_dir='./bert_fine_tuned_model',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.001,
    logging_dir='./logs',
    logging_steps=50,
    save_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=100,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics

)

In [ ]:
# Fine-tune the model
trainer.train()

In [ ]:
# Evaluate the model on the validation set
results = trainer.evaluate()
print(results)
print("F1 Score:", results['eval_f1'])
print("Accuracy:", results['eval_accuracy'])